In [1]:
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import warnings
from IPython.display import clear_output
from multiprocessing import Pool
from time import time
warnings.filterwarnings('ignore')

In [2]:
def get_time_spaced_traindata(data):
    data['delta_t'] = np.int64((data.index - data.index[0]).total_seconds() // 3600) + 1
#     data = data.drop(columns=['station_id'])
#     data.index = range(len(data))
    return data

def get_time_spaced_testdata(data):
    data['delta_t'] = np.int64((data.index - data.index[0]).total_seconds() // 3600) + 1  
#     data = data.drop(columns=['station_id'])
#     data.index = range(len(data))
    aq_data = data['PM25_Concentration']
    data.drop(columns=['PM25_Concentration'])
    return data,aq_data

In [3]:
fold=2 # not using for loop to avoid ram overflow
print('fold', fold)

train_data = pd.read_csv('../data/processed/fold_'+str(fold)+'_train_mar.csv.gz')
train_data['time'] = pd.to_datetime(train_data['time'])
train_data = train_data.set_index('time').sort_values(['time', 'station_id'])

test_data = pd.read_csv('../data/processed/fold_'+str(fold)+'_test_mar.csv.gz')
test_data['time'] = pd.to_datetime(test_data['time'])
test_data = test_data.set_index('time').sort_values(['time', 'station_id'])

fold 2


In [4]:
train_input = get_time_spaced_traindata(train_data['2015-03-01':'2015-03-31'])
test_input,test_output = get_time_spaced_testdata(test_data['2015-03-01':'2015-03-31'])

In [5]:
display(train_input,test_input,test_output)

,station_id,longitude,latitude,PM25_Concentration,temperature,humidity,wind_speed,filled,weather_0.0,weather_1.0,...,wind_direction_1.0,wind_direction_2.0,wind_direction_3.0,wind_direction_4.0,wind_direction_9.0,wind_direction_13.0,wind_direction_14.0,wind_direction_23.0,wind_direction_24.0,delta_t
time,,,,,,,,,,,,,,,,,,,,,
2015-03-01 00:00:00,1002,116.205310,40.003950,26.0,3.0,23.0,3.00,False,0,0,...,0,0,0,0,0,0,1,0,0,1
2015-03-01 00:00:00,1003,116.184239,39.914409,21.0,4.0,17.0,8.50,False,0,0,...,0,0,0,0,0,0,0,0,1,1
2015-03-01 00:00:00,1004,116.171150,39.815128,25.0,4.0,18.0,3.00,False,0,0,...,0,0,0,0,0,0,0,0,1,1
2015-03-01 00:00:00,1005,116.136045,39.742767,25.0,2.0,26.0,4.68,False,0,1,...,0,0,0,0,0,0,0,1,0,1
2015-03-01 00:00:00,1006,116.287451,39.987313,30.0,3.0,23.0,3.00,False,0,0,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-03-31 23:00:00,1031,115.988000,40.365000,123.0,7.5,83.0,5.22,False,0,1,...,0,0,0,0,0,1,0,0,0,744
2015-03-31 23:00:00,1032,116.911000,40.499000,77.0,5.5,86.5,1.26,False,0,1,...,0,0,0,0,0,0,1,0,0,744
2015-03-31 23:00:00,1033,117.120000,40.100000,62.0,6.5,89.5,1.08,False,0,1,...,0,0,0,0,0,0,0,0,0,744


,station_id,longitude,latitude,PM25_Concentration,temperature,humidity,wind_speed,filled,weather_0.0,weather_1.0,...,wind_direction_1.0,wind_direction_2.0,wind_direction_3.0,wind_direction_4.0,wind_direction_9.0,wind_direction_13.0,wind_direction_14.0,wind_direction_23.0,wind_direction_24.0,delta_t
time,,,,,,,,,,,,,,,,,,,,,
2015-03-01 00:00:00,1008,116.348991,39.954047,68.0,3.000000,23.0,3.00,False,0,0,...,0,0,0,0,0,0,1,0,0,1
2015-03-01 00:00:00,1012,116.460742,39.937119,67.0,1.000000,29.0,14.04,False,0,0,...,0,0,0,0,0,0,0,0,1,1
2015-03-01 00:00:00,1016,116.279082,39.863470,28.0,4.000000,18.0,3.00,False,0,0,...,0,0,0,0,0,0,0,0,1,1
2015-03-01 00:00:00,1017,116.406155,39.718147,42.0,3.000000,26.0,4.68,False,0,1,...,0,0,0,0,0,0,0,0,1,1
2015-03-01 00:00:00,1022,116.443448,39.920993,52.0,1.000000,29.0,14.04,False,0,0,...,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-03-31 23:00:00,1024,116.230000,40.216999,79.0,9.666667,77.0,3.00,False,0,1,...,0,0,0,0,0,0,1,0,0,744
2015-03-31 23:00:00,1026,117.099999,40.143000,79.0,6.500000,89.5,1.08,False,0,1,...,0,0,0,0,0,0,0,0,0,744
2015-03-31 23:00:00,1027,116.628000,40.328000,83.0,8.000000,83.0,1.44,False,0,1,...,0,0,0,0,0,0,0,0,1,744


time
2015-03-01 00:00:00    68.0
2015-03-01 00:00:00    67.0
2015-03-01 00:00:00    28.0
2015-03-01 00:00:00    42.0
2015-03-01 00:00:00    52.0
                       ... 
2015-03-31 23:00:00    79.0
2015-03-31 23:00:00    79.0
2015-03-31 23:00:00    83.0
2015-03-31 23:00:00    83.0
2015-03-31 23:00:00    73.0
Name: PM25_Concentration, Length: 7440, dtype: float64

In [6]:
display(train_input.shape, test_input.shape, test_output.shape)

(14880, 35)

(7440, 35)

(7440,)

In [16]:
train_input.columns[[i for i in range(len(train_input.columns)) if train_input.columns[i] not in ['station_id', 'filled', 'PM25_Concentration']]]

Index(['longitude', 'latitude', 'temperature', 'humidity', 'wind_speed',
       'weather_0.0', 'weather_1.0', 'weather_2.0', 'weather_3.0',
       'weather_4.0', 'weather_5.0', 'weather_6.0', 'weather_7.0',
       'weather_8.0', 'weather_9.0', 'weather_11.0', 'weather_12.0',
       'weather_13.0', 'weather_14.0', 'weather_15.0', 'weather_16.0',
       'wind_direction_0.0', 'wind_direction_1.0', 'wind_direction_2.0',
       'wind_direction_3.0', 'wind_direction_4.0', 'wind_direction_9.0',
       'wind_direction_13.0', 'wind_direction_14.0', 'wind_direction_23.0',
       'wind_direction_24.0', 'delta_t'],
      dtype='object')

In [7]:
train_input.reset_index().to_csv('../data/time_feature/fold'+str(fold)+'/train_data_mar.csv.gz', index=None)
test_input.reset_index().to_csv('../data/time_feature/fold'+str(fold)+'/test_data_mar.csv.gz', index=None)
test_output.reset_index().to_csv('../data/time_feature/fold'+str(fold)+'/test_output_mar.csv.gz', index=None)